In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_ur5e_and_table_to_builder
from ompl import geometric as og
from cloth_tools.ompl.single_arm_planner import SingleArmOmplPlanner

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_index, gripper_index = add_ur5e_and_table_to_builder(robot_diagram_builder)

In [ ]:
diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
ompl_planner = SingleArmOmplPlanner(is_state_valid_fn=collision_checker.CheckConfigCollisionFree)

In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
goal_joints = np.deg2rad([180, -135, 95, -50, -90, -90])

ompl_planner._set_start_and_goal_configurations(start_joints, goal_joints)

In [ ]:
simple_setup = ompl_planner._simple_setup
space_info = simple_setup.getSpaceInformation()

In [ ]:
from ompl import tools as ot

benchmark = ot.Benchmark(simple_setup, "twist_base_90")
benchmark.addPlanner(og.RRT(space_info))
benchmark.addPlanner(og.RRTstar(space_info))
benchmark.addPlanner(og.RRTConnect(space_info))
benchmark.addPlanner(og.TRRT(space_info))
benchmark.addPlanner(og.LazyRRT(space_info))
benchmark.addPlanner(og.KPIECE1(space_info))
benchmark.addPlanner(og.LBKPIECE1(space_info))

request = ot.Benchmark.Request()
request.runCount = 5
request.maxTime = 10.0

In [ ]:
benchmark.benchmark(request)

In [ ]:
print(benchmark.results())

In [ ]:
benchmark.saveResultsToFile()

Now use the `ompl_benchmark_statistics.py` script to generate a database file that we can upload to plannerarena.org:

```
python ompl_benchmark_statistics.py "ompl_idlab185_2024-01-23 11:12:48.log" benchmark.db
```